### Welcome to the ProtoSyn.jl examples

# 6 - Steepest Descent

ProtoSyn makes available the Steepest Descent Driver, whose simulation algorithm calculates the forces being felt on each atom of a molecular structure (via an Energy Function instance) and updates the atoms position in accordance, as to relax the structure. In this example, we will relax the 2A3D peptide using the Steepest Descent Driver.

In [2]:
using ProtoSyn

ProtoSyn will now only display error messages. Debug: ⨯ | Info: ⨯ | Warnings: ⨯ | Errors: ✓
[ Loading: External packages
 | Loading SIMD
 | Loading CUDA
[ Loading: Setting up global variables
 | Current acceleration set to ProtoSyn.CUDA_2
[ Loading: Core module
 | Loading Calculators
 | Loading Mutators
 | Loading Drivers
[ Loading: Peptides module
[ Loading: Materials module
[ Loading: Sugars module
[ Loading: Common module
[ Loading: External models
 | Loading TorchANI
 | Loading ONNX models
 | Loading SeqDes
[ Loading: ProtoSyn loaded successfully!

.      ____            _       ____              
      |  _ \ _ __ ___ | |_ ___/ ___| _   _ _ __  
      | |_) | '__/ _ \| __/ _ \___ \| | | | '_ \ 
      |  __/| | | (_) | || (_) |__) | |_| | | | |
      |_|   |_|  \___/ \__\___/____/ \__, |_| |_|
                                       |_/       
    
      ---------------------------------------------

 Version      : 1.10
 License      : GNU-GPL-3
 Developed by : José Pereira (jose.m

┌ Info: Precompiling ProtoSyn [c9758760-7c0d-11e9-0ffc-fb9355b7d293]
└ @ Base loading.jl:1423


1. Load the 2A3D structure.

The assignment of the default atom names and charges is not actually necessary for this example, but it's a good practice for when electrostatics are being calculated.

In [3]:
pose = ProtoSyn.Peptides.load("data/2a3d.pdb")
ProtoSyn.Peptides.assign_default_atom_names!(pose)
ProtoSyn.Peptides.Calculators.Electrostatics.assign_default_charges!(pose);

2. Define the energy function

For this example, we'll minimize the TorchANI ML model energy (while maintaining a bond distance restraint). Other components, such as long-range electrostatics or all-atom clash restraints, can be added when necessary.

In [4]:
torchani        = ProtoSyn.Calculators.TorchANI.get_default_torchani_model(α = 1.0)
bond_distance   = ProtoSyn.Calculators.Restraints.get_default_bond_distance_restraint(α = 1.0)
energy_function = ProtoSyn.Calculators.EnergyFunction([torchani, bond_distance])

🗲  Energy Function (2 components):
+----------------------------------------------------------------------+
| Index | Component name                                | Weight (α)   |
+----------------------------------------------------------------------+
| 1     | TorchANI_ML_Model                             |       1.00   |
| 2     | Bond_Distance_Rest                            |       1.00   |
+----------------------------------------------------------------------+
 ● Update forces: false
 ● Selection: Set
 └── TrueSelection (Atom)


3. Evaluate the initial state

In [9]:
energy_function(pose)
ProtoSyn.write(pose, "output/example6.pdb")
display(pose.state.e)

Dict{Symbol, Float64} with 3 entries:
  :Total              => -11.8251
  :TorchANI_ML_Model  => -11.8251
  :Bond_Distance_Rest => 0.0

4. Generate an alternative starting point

For the purpose of this example, the Backrub Mutator will quickly be employed to gently randomize the position of the atoms, in order to more visually veriy the energy minimization process. 

In [10]:
scramble! = ProtoSyn.Mutators.BackrubMutator(ProtoSyn.rand_vector_in_sphere, 1.0, 0.2, nothing)
scramble!(pose)
energy_function(pose)
ProtoSyn.append(pose, "output/example6.pdb")
display(pose.state.e)

Dict{Symbol, Float64} with 3 entries:
  :Total              => 27.1873
  :TorchANI_ML_Model  => 16.0743
  :Bond_Distance_Rest => 11.1131

5. Generate and run a Steepest Descent Driver

In [11]:
callback = ProtoSyn.Common.default_energy_step_frame_callback(1, "output/example6.pdb")
steepest_descent = ProtoSyn.Drivers.SteepestDescent(energy_function, callback, 500, 0.001, 0.1)

⚒  Steepest Descent Driver:
 ├──  ●  Evaluator:
 |    └── 🗲  Energy Function (2 components):
 |        +----------------------------------------------------------------------+
 |        | Index | Component name                                | Weight (α)   |
 |        +----------------------------------------------------------------------+
 |        | 1     | TorchANI_ML_Model                             |       1.00   |
 |        | 2     | Bond_Distance_Rest                            |       1.00   |
 |        +----------------------------------------------------------------------+
 |         ● Update forces: false
 |         ● Selection: Set
 |         └── TrueSelection (Atom)
 |   
 ├──  ● Callback:
 |    └── ✉  Callback:
 |        +----------------------------------------------------------------------+
 |        | Index | Field                     | Value                            |
 |        +----------------------------------------------------------------------+
 |        | 1  

In [8]:
steepest_descent(pose)

       0      29.466
       1      14.519
       2       2.827
       3      -4.953
       4      -7.762
       5      -8.613
       6      -9.353
       7      -9.644
       8     -10.012
       9     -10.207
      10     -10.386
      11     -10.482
      12     -10.597
      13     -10.654
      14     -10.743
      15     -10.815
      16     -10.830
      17     -10.916
      18     -10.916
      19     -10.949
      20     -10.964
      21     -10.990
      22     -11.002
      23     -11.024
      24     -11.034
      25     -11.052
      26     -11.061
      27     -11.077
      28     -11.084
      29     -11.097
      30     -11.105
      31     -11.116
      32     -11.124
      33     -11.132
      34     -11.142
      35     -11.149
      36     -11.159
      37     -11.165
      38     -11.175
      39     -11.180
      40     -11.189
      41     -11.194
      42     -11.203
      43     -11.208
      44     -11.217
      45     -11.220
      46     -11.229
      47     

Pose{Topology}(Topology{/2a3d:1786}, State{Float64}:
 Size: 1140
 i2c: false | c2i: false
 Energy: Dict(:Total => -11.825096130371094, :TorchANI_ML_Model => -11.825096130371094, :Bond_Distance_Rest => 0.0)
)

## Conclusion

In this quick example we explored the usage of the Steepest Descent Driver to relax a structure based on the forces being felt. This is usefull as a quick way to relax angles and bond distances in simulations, as well as fine tune the packaging of sidechains.